In [62]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

In [63]:
yolo = cv.dnn.readNet(".\yolov3.weights", ".\yolov3.cfg")

In [64]:
classess = []
with open(".\coco.names", 'r') as f:
    classess = f.read().splitlines()
    

In [65]:
cap = cv.VideoCapture(r"D:\Object detection\video.mp4")

if not cap.isOpened():
    print("Error opening Video File.")



In [66]:
fps = cap.get(cv.CAP_PROP_FPS)
print(f"Frames per second: {fps}")

Frames per second: 25.0


In [67]:
cv.setUseOptimized(True)
cv.setNumThreads(4) 

In [68]:
while (True):
    ret, frame = cap.read()
    height, width,_ = frame.shape
    
    blob = cv.dnn.blobFromImage(frame, 1/255, (320, 320), (0,0,0), swapRB=True, crop=False)
    i = blob[0].reshape(320, 320, 3)
    
    yolo.setInput(blob)
    output_layers_name = yolo.getUnconnectedOutLayersNames()
    layeroutput = yolo.forward(output_layers_name)
    
    boxes = []
    confidences = []
    class_ids = []

    for output in layeroutput:
        for detection in output:
            score = detection[5:]
            class_id = np.argmax(score)
            confidence = score[class_id]
            if confidence > 0.7:
                center_x = int(detection[0]*width)
                center_y = int(detection[1]*height)
                w = int(detection[2]*width)
                h = int(detection[3]*height)
                
                x = int(center_x - w/2)
                y = int(center_y - h/2)
                
                boxes.append([x,y,w,h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    
    
    indexes = cv.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    font = cv.FONT_HERSHEY_PLAIN
    colors = np.random.uniform(0, 255, size = (len(boxes), 3))
    
    for i in indexes.flatten():
        x,y,w,h = boxes[i]
        label = str(classess[class_ids[i]])
        confi = str(round(confidences[i], 2))
        color = colors[i]
        
        cv.rectangle(frame, (x,y), (x+w, y+h), color, 3)
        cv.putText(frame, label+""+confi, (x, y+20), font, 2, (255, 255, 255), 2)  
    

    cv.imshow("output", frame)
    
    if (cv.waitKey(1) & 0xFF == ord('q')):
        break

cap.release()
cv.destroyAllWindows()